In [6]:
import praw
import json
from datetime import datetime
from confluent_kafka import Producer

In [7]:
user_agent = "Reddit_Scrapper 1.0 by / u//FeatureChoice5036"
reddit = praw.Reddit(
    client_id="zb_c3ATW7jp24qk_phIfbg",
    client_secret="6xTIjpqqxwo0E2UC8DiYId_YjAqv4Q",
    user_agent=user_agent
)

In [8]:
def get_tweets(keywords):
    tweets = []
    for keyword in keywords:
        for submission in reddit.subreddit(keyword).hot(limit=None):
            headlines = {}
            headlines['id'] = submission.id
            headlines['value'] = submission.title
            headlines['date'] = datetime.utcfromtimestamp(submission.created_utc).strftime("%Y-%m-%d %H:%M:%S")
            headlines['topic'] = keyword
            tweets.append(headlines)
    return tweets

In [9]:
def send_tweets(Tweets):
    bootstrap_servers = "localhost:9092"
    producer = Producer({'bootstrap.servers': bootstrap_servers})
    for tweet in Tweets:
        json_data = json.dumps(tweet)
        producer.produce('reddit_postgres', value=json_data.encode('utf-8'))
        print(tweet)
    producer.flush()

In [10]:
Keywords = ["Naruto", "Onepiece"]

In [ ]:
while True:
    tweets = get_tweets(Keywords)
    send_tweets(tweets)